# PSからPLを制御するためのプログラム

In [99]:
from pynq import PL
PL.reset()

## FPGA書き込み

In [100]:
from pynq import Overlay

ol = Overlay("kmj_gen.bit")

## IPコア定義

In [101]:
dma = ol.axi_dma_0
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

In [102]:
from pynq import MMIO
top_mmio = MMIO(ol.ip_dict['top_0']['phys_addr'], ol.ip_dict['top_0']['addr_range'])

## AXI Stream でデータを書き込み

In [103]:
from pynq import allocate
import numpy as np

data_size = 10
input_buffer = allocate(shape=(data_size,), dtype=np.uint8)
output_buffer = allocate(shape=(data_size,), dtype=np.uint8)

In [104]:
for i in range(data_size):
    input_buffer[i] = i

In [105]:
input_buffer, output_buffer

(PynqBuffer([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [117]:
# データ送信
dma_send.transfer(input_buffer)

## AXI LITE で回路全体を制御

In [118]:
top_mmio.write(0, 0)

In [119]:
# slv_reg0[2:0] : {set, run, rst_n}
top_mmio.write(0, 3)

# slv_reg1[1:0] : mode
top_mmio.write(4, 0b00)

In [120]:
# slv_reg2[0] : finish
top_mmio.read(8)

1

## AXI Stream でデータを読み込み

In [110]:
dma_recv.transfer(output_buffer)

In [111]:
output_buffer

PynqBuffer([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)